In [5]:
import requests
# from sqlalchemy import create_engine
import pyodbc
import pandas as pd
# import os
import numpy as np

C:\Users\mipec\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (5.1.0)/charset_normalizer (2.0.10) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
df = pd.read_csv('./listaprzystanków.csv', index_col=0)
df = df.drop(columns = ['Przystanek'])
df.columns = ['nazwa']
df['numer'] = df['nazwa'].apply(lambda x: x.strip(" ").split(" ")[-1])
df['nazwa'] = df['nazwa'].apply(lambda x: x[:-5])
df

,nazwa,numer
1,11 LISTOPADA,01
2,11 LISTOPADA,02
3,17 STYCZNIA,01
4,17 STYCZNIA,02
5,17 STYCZNIA,04
...,...,...
4889,ZYNDRAMA Z MASZKOWIC,02
4890,ŻÓŁKIEWSKIEGO,01
4891,ŻÓŁKIEWSKIEGO,02
4892,ŻÓŁKIEWSKIEGO,03


In [99]:
def connect():
    # TODO: zamień na swoje :)))
    server = 'DESKTOP-V5DA5NT'
    database = 'HDiSBI_projekt'
    username = 'sa'
    password = 'mipecki123'
    connection_string = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'

    # Nawiązanie połączenia do bazy danych
    conn = pyodbc.connect(connection_string)
    return(conn)


def check_for_table(conn, table_name, names):
    cursor = conn.cursor()
    names = [name.replace(" ", "_") for name in names]
    names = " varchar(255),\n".join(names) + " varchar(255)"
    # check if the table exists
    if cursor.tables(table=table_name, tableType='TABLE').fetchone():
        # query = f"DROP TABLE {table_name}"
        # cursor.execute(query)
        return
    # create the table if it does not exist
    query = f"CREATE TABLE {table_name} ({names})".format(
            names = names
        )

    print(query)
    cursor.execute(query)
    print(f"Table {table_name} created successfully.")

    # close the cursor and connection
    conn.commit()
    cursor.close()
    return


# TODO: kluczyk też możesz zmienić jeśli chcesz :))
apikey = '9e511e7a-be05-47fe-a578-34190f4bfe2c'

In [7]:
dictionaries = []
maksik = len(np.unique(df['nazwa']))
for i, nazwaprzystanku in enumerate(np.unique(df['nazwa'])):
    if i % int(maksik/10) == 0:
        print(f'we are at the {int(i*100/maksik)}th percentile')
    url1 = 'https://api.um.warszawa.pl/api/action/dbtimetable_get?id=b27f4c17-5c50-4a5b-89dd-236b282bc499&name=' + nazwaprzystanku + '&apikey=' + apikey


    ### dla url 1
    response = requests.get(url1)
    if response.status_code == 200:
    # Odpowiedź jest udana, można przetwarzać dane
        data = response.json()  # Jeśli odpowiedź jest w formacie JSON
    try:
        for i in range(len(data['result'])):
            dictionary = {}
            for j in range(len(data['result'][i]['values'])):
                # print(data['result'][i]['values'][j]['key'], data['result'][i]['values'][j]['value'])
                c = data['result'][i]['values'][j]['key']
                v = data['result'][i]['values'][j]['value']
                dictionary[c] = v
            dictionaries.append(dictionary)
    except:
        continue
stopcode = pd.DataFrame(dictionaries)


conn = connect()
table_name = "Stopcode"
check_for_table(conn, table_name, stopcode.columns)
cursor = conn.cursor()
# Insert Dataframe into SQL Server:
for index, row in stopcode.iterrows():
    cursor.execute(f"INSERT INTO {table_name} (Zespol,Nazwa) values(?,?)",
                (row.zespol, row.nazwa))
conn.commit()
cursor.close()

we are at the 0.0th percentile
we are at the 0.5232862375719518th percentile
we are at the 1.0465724751439036th percentile
we are at the 1.5698587127158556th percentile
we are at the 2.0931449502878072th percentile
we are at the 2.6164311878597593th percentile
we are at the 3.1397174254317113th percentile
we are at the 3.663003663003663th percentile
we are at the 4.1862899005756145th percentile
we are at the 4.7095761381475665th percentile
we are at the 5.2328623757195185th percentile
we are at the 5.7561486132914705th percentile
we are at the 6.279434850863423th percentile
we are at the 6.802721088435374th percentile
we are at the 7.326007326007326th percentile
we are at the 7.849293563579278th percentile
we are at the 8.372579801151229th percentile
we are at the 8.895866038723181th percentile
we are at the 9.419152276295133th percentile
we are at the 9.942438513867085th percentile
we are at the 10.465724751439037th percentile
we are at the 10.989010989010989th percentile
we are at th

In [11]:
dictionaries = []
zespoly = list(stopcode['zespol'].unique())
k = 0.05
for i, zespol in enumerate(zespoly):
    name = list(stopcode[stopcode['zespol'] == zespol].nazwa)[0]

    if i > k * len(zespoly):
        print(f'{k*100}th centile')
        k += 0.05

    
    for busstopNr in list(df[df['nazwa'] == name.upper()].numer):
        
        url2 = 'https://api.um.warszawa.pl/api/action/dbtimetable_get?id=88cd555f-6f31-43ca-9de4-66c479ad5942&busstopId=' + zespol + '&busstopNr=' + busstopNr + '&apikey=' + apikey

        ### dla url 2
        response = requests.get(url2)
        if response.status_code == 200:
            # Odpowiedź jest udana, można przetwarzać dane
            data = response.json()  # Jeśli odpowiedź jest w formacie JSON
            # print(data)
        for i in range(len(data['result'])):
            dictionary = {"zespol" : zespol,
                        "stop nr" : busstopNr}
            for j in range(len(data['result'][i]['values'])):
                # print(data['result'][i]['values'][j]['key'], data['result'][i]['values'][j]['value'])
                c = data['result'][i]['values'][j]['key']
                v = data['result'][i]['values'][j]['value']
                dictionary[c] = v
            dictionaries.append(dictionary)
stopinfo = pd.DataFrame(dictionaries)


conn = connect()
check_for_table(conn, table_name, stopinfo.columns)
cursor = conn.cursor()
# Insert Dataframe into SQL Server:
for index, row in stopinfo.iterrows():
    cursor.execute("INSERT INTO Stopinfo (zespol, stop_nr, linia) values(?,?,?)",
                    (row['zespol'], row['stop nr'], row.linia))
conn.commit()
cursor.close()
conn.close()

In [46]:
# tmp = stopinfo.merge(stopcode, how='inner', on = 'zespol')
# tmp
# dictionaries = []
# k=0.01

# # for i, row in tmp.iterrows():
# #     zespol = row.zespol
# #     slupek = row['stop nr']
# #     linia = row['linia']
# #     if i / len(tmp) > k:
# #         print(f"{k*100}th percentile")
# #         k+=0.01
# #     if type(slupek) == tuple:
# #         slupek = slupek[0]

# #     url3 = 'https://api.um.warszawa.pl/api/action/dbtimetable_get?id=e923fa0e-d96c-43f9-ae6e-60518c9f3238&busstopId=' + str(zespol) + '&busstopNr=' + slupek + '&line=' + str(linia) + '&apikey=' + apikey

# #     ### dla url 3
# #     response = requests.get(url3)
# #     if response.status_code == 200:
# #         # Odpowiedź jest udana, można przetwarzać dane
# #         data = response.json()  # Jeśli odpowiedź jest w formacie JSON
# #     dictionaries = []
# #     for i in range(len(data['result'])):
# #         dictionary = {'zespol' : zespol,
# #                     "slupek" : slupek,
# #                     'linia' : linia}
# #         for j in range(len(data['result'][i]['values'])):
# #             # print(data['result'][i]['values'][j]['key'], data['result'][i]['values'][j]['value'])
# #             c = data['result'][i]['values'][j]['key']
# #             v = data['result'][i]['values'][j]['value']
# #             dictionary[c] = v
# #         dictionaries.append(dictionary)
# # schedule = pd.DataFrame(dictionaries)


# conn = connect()
# check_for_table(conn, 'Schedules', schedule.columns)
# cursor = conn.cursor()
# # Insert Dataframe into SQL Server:
# for index, row in schedule.iterrows():

#     cursor.execute("INSERT INTO Schedules (zespol, slupek, linia, symbol_2, symbol_1, brygada, kierunek, trasa, czas) values(?,?,?,?,?,?,?,?,?)",
#         (row['zespol'], row['slupek'], row.linia, row.symbol_2, row.symbol_1, row.brygada, row.kierunek, row.trasa, row.czas))
# cursor.close()
# conn.close()
# schedule

KeyboardInterrupt: 

In [78]:
tmp = stopinfo.merge(stopcode, how='inner', on = 'zespol')
dictionaries = []
compare=0.01
for i, row in tmp.iterrows():
    if i/16494 > compare:
        print(f"{i} out of {16494} rows\nschedules is estimated to be of length {16494/i * len(dictionaries)}")
        compare+=0.01
    zespol = row.zespol
    slupek = row['stop nr']
    linia = row['linia']
    if type(slupek) == tuple:
        slupek = slupek[0]

    url3 = 'https://api.um.warszawa.pl/api/action/dbtimetable_get?id=e923fa0e-d96c-43f9-ae6e-60518c9f3238&busstopId=' + str(zespol) + '&busstopNr=' + slupek + '&line=' + str(linia) + '&apikey=' + apikey

    ### dla url 3
    response = requests.get(url3)
    if response.status_code == 200:
        # Odpowiedź jest udana, można przetwarzać dane
        data = response.json()  # Jeśli odpowiedź jest w formacie JSON
    for k in range(len(data['result'])):
        dictionary = {'zespol' : zespol,
                    "slupek" : slupek,
                    'linia' : linia}
        for j in range(len(data['result'][k]['values'])):
            # print(data['result'][i]['values'][j]['key'], data['result'][i]['values'][j]['value'])
            c = data['result'][k]['values'][j]['key']
            v = data['result'][k]['values'][j]['value']
            dictionary[c] = v
        dictionaries.append(dictionary)
schedule = pd.DataFrame(dictionaries)

165 out of 16494 rows
schedules is estimated to be of length 669156.5818181819
330 out of 16494 rows
schedules is estimated to be of length 764471.9090909092
495 out of 16494 rows
schedules is estimated to be of length 755458.5212121212
660 out of 16494 rows
schedules is estimated to be of length 792336.7727272727
825 out of 16494 rows
schedules is estimated to be of length 811904.6545454545
990 out of 16494 rows
schedules is estimated to be of length 803307.7818181817
1155 out of 16494 rows
schedules is estimated to be of length 782415.3818181818
1320 out of 16494 rows
schedules is estimated to be of length 782715.2727272727
1485 out of 16494 rows
schedules is estimated to be of length 787902.2747474747
1650 out of 16494 rows
schedules is estimated to be of length 771659.2945454546
1815 out of 16494 rows
schedules is estimated to be of length 767511.7123966941
1980 out of 16494 rows
schedules is estimated to be of length 746111.9212121211
2145 out of 16494 rows
schedules is estimated 

In [101]:
# TODO: tu też zamień na swoje
server = 'DESKTOP-V5DA5NT'
database = 'HDiSBI_projekt'
username = 'sa'
password = 'mipecki123'
connection_string = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'

# Nawiązanie połączenia do bazy danych
conn = pyodbc.connect(connection_string)

check_for_table(conn, 'Schedules', schedule.columns)
cursor = conn.cursor()
k=0.01
# Insert Dataframe into SQL Server:
for index, row in schedule.iterrows():
    if index/717139 >= k:
        k+=0.01
        print(f"{index} out of 717140")
    cursor.execute("INSERT INTO Schedules (zespol, slupek, linia, symbol_1, symbol_2, brygada, kierunek, trasa, czas) values(?,?,?,?,?,?,?,?,?)",
        (row['zespol'], row['slupek'], row.linia, row.symbol_1, row.symbol_2, row.brygada, row.kierunek, row.trasa, row.czas))
conn.commit()
cursor.close()
conn.close()
schedule

7172 out of 717140
14343 out of 717140
21515 out of 717140
28686 out of 717140
35857 out of 717140
43029 out of 717140
50200 out of 717140
57372 out of 717140
64543 out of 717140
71714 out of 717140
78886 out of 717140
86057 out of 717140
93229 out of 717140
100400 out of 717140
107571 out of 717140
114743 out of 717140
121914 out of 717140
129086 out of 717140
136257 out of 717140
143428 out of 717140
150600 out of 717140
157771 out of 717140
164942 out of 717140
172114 out of 717140
179285 out of 717140
186457 out of 717140
193628 out of 717140
200799 out of 717140
207971 out of 717140
215142 out of 717140
222314 out of 717140
229485 out of 717140
236656 out of 717140
243828 out of 717140
250999 out of 717140
258171 out of 717140
265342 out of 717140
272513 out of 717140
279685 out of 717140
286856 out of 717140
294027 out of 717140
301199 out of 717140
308370 out of 717140
315542 out of 717140
322713 out of 717140
329884 out of 717140
337056 out of 717140
344227 out of 717140
351399

,zespol,slupek,linia,symbol_2,symbol_1,brygada,kierunek,trasa,czas
0,4009,01,154,null,null,1,rondo Zesłańców Syberyjskich,TP-RZS,04:27:00
1,4009,01,154,null,null,4,rondo Zesłańców Syberyjskich,TP-RZS,04:57:00
2,4009,01,154,null,null,05,rondo Zesłańców Syberyjskich,TP-RZS,05:17:00
3,4009,01,154,null,null,1,rondo Zesłańców Syberyjskich,TP-RZS,05:37:00
4,4009,01,154,null,null,06,rondo Zesłańców Syberyjskich,TP-RZS,05:57:00
...,...,...,...,...,...,...,...,...,...
717135,2012,04,24,null,null,9,Gocławek,TP-GCW,22:15:00
717136,2012,04,24,null,null,12,Gocławek,TP-GCW,22:35:00
717137,2012,04,24,null,null,2,Gocławek,TP-GCW,22:55:00
717138,2012,04,24,null,null,3,Gocławek,TP-GCW,23:16:00


In [102]:
# aktualna pozycja pojazdów
dictionaries = []
for linia in tmp['linia'].unique():

    url = f"https://api.um.warszawa.pl/api/action/busestrams_get/?resource_id=f2e5503e927d-4ad3-9500-4ab9e55deb59&apikey={apikey}&type=1&line={linia}"
    response = requests.get(url)
    if response.status_code == 200:
        # Odpowiedź jest udana, można przetwarzać dane
        data = response.json()  # Jeśli odpowiedź jest w formacie JSON
        print(data)
    for i in range(len(data['result'])):
        dictionary = {"linia" : linia}
        # print(data['result'][i]['values'][j]['key'], data['result'][i]['values'][j]['value'])
        for c, v in data['result'][i].items():
            dictionary[c] = v
        dictionaries.append(dictionary)


currentposition = pd.DataFrame(dictionaries)
currentposition

{'result': [{'Lines': '154', 'Lon': 20.92092, 'VehicleNumber': '4222', 'Time': '2023-05-15 22:10:36', 'Lat': 52.1967506, 'Brigade': '3'}]}
{'result': [{'Lines': '228', 'Lon': 20.920278, 'VehicleNumber': '1053', 'Time': '2023-05-15 22:37:36', 'Lat': 52.196841, 'Brigade': '2'}, {'Lines': '228', 'Lon': 20.921331, 'VehicleNumber': '1056', 'Time': '2023-05-15 23:05:09', 'Lat': 52.196923, 'Brigade': '3'}, {'Lines': '228', 'Lon': 20.918503, 'VehicleNumber': '1059', 'Time': '2023-05-15 23:41:11', 'Lat': 52.196801, 'Brigade': '1'}]}
{'result': [{'Lines': '328', 'Lon': 20.9951692, 'VehicleNumber': '1029', 'Time': '2023-05-15 20:50:30', 'Lat': 52.1883549, 'Brigade': '03'}, {'Lines': '328', 'Lon': 20.91863, 'VehicleNumber': '1050', 'Time': '2023-05-15 22:30:27', 'Lat': 52.196795, 'Brigade': '01'}, {'Lines': '328', 'Lon': 20.921203, 'VehicleNumber': '1051', 'Time': '2023-05-15 23:03:20', 'Lat': 52.196914, 'Brigade': '02'}]}
{'result': [{'Lines': 'N38', 'Lon': 20.9585667, 'VehicleNumber': '4337', 'T

,linia,Lines,Lon,VehicleNumber,Time,Lat,Brigade
0,154,154,20.920920,4222,2023-05-15 22:10:36,52.196751,3
1,228,228,20.920278,1053,2023-05-15 22:37:36,52.196841,2
2,228,228,20.921331,1056,2023-05-15 23:05:09,52.196923,3
3,228,228,20.918503,1059,2023-05-15 23:41:11,52.196801,1
4,328,328,20.995169,1029,2023-05-15 20:50:30,52.188355,03
...,...,...,...,...,...,...,...
773,L29,L29,20.499007,60055,2023-05-15 23:48:52,52.261019,1
774,L15,L15,21.034847,10074,2023-05-15 20:13:26,52.083645,52
775,L21,L21,21.190642,70032,2023-05-15 20:34:02,51.987455,1
776,L28,L28,21.126193,70043,2023-05-15 20:25:24,52.022716,51


In [103]:
conn = connect()
currentposition = currentposition.drop(columns=['Lines'])
check_for_table(conn, 'CurrentPosition', currentposition.columns)
cursor = conn.cursor()
# Insert Dataframe into SQL Server:
for index, row in currentposition.iterrows():
    cursor.execute("INSERT INTO CurrentPosition (linia, Lon, Lat, VehicleNumber, Brigade, Time) values(?,?,?,?,?,?)",
        (row['linia'], row['Lon'], row.Lat, row.VehicleNumber, row.Brigade, row.Time))
conn.commit()
cursor.close()
conn.close()

CREATE TABLE CurrentPosition (linia varchar(255),
Lon varchar(255),
VehicleNumber varchar(255),
Time varchar(255),
Lat varchar(255),
Brigade varchar(255))
Table CurrentPosition created successfully.


In [104]:
# trasy pojazdów routes

url = f"https://api.um.warszawa.pl/api/action/public_transport_routes/?apikey={apikey}"
dictionaries = []

response = requests.get(url)
if response.status_code == 200:
    # Odpowiedź jest udana, można przetwarzać dane
    data = response.json()  # Jeśli odpowiedź jest w formacie JSON
    for linia in data['result'].keys():
        for key in data['result'][linia].keys():
            for key2 in  data['result'][linia][key].keys():
                dictionary = {'linia' : linia,
                              'kod' : key,
                              'nr przystanku na trasie' : key2}
                
                for c, v in data['result'][linia][key][key2].items():
                    dictionary[c] = v
                dictionaries.append(dictionary)
    # print(data['result'][i]['values'][j]['key'], data['result'][i]['values'][j]['value'])  
routes = pd.DataFrame(dictionaries)
routes = routes.rename(columns={'nr przystanku na trasie' : 'enroute_nr'})
routes

,linia,kod,nr przystanku na trasie,odleglosc,ulica_id,nr_zespolu,typ,nr_przystanku
0,217,TX-MIM,11,4387,0150,3354,1,51
1,217,TX-MIM,10,3887,0150,3353,1,01
2,217,TX-MIM,13,5396,0271,3468,1,02
3,217,TX-MIM,12,4941,0271,3385,1,04
4,217,TX-MIM,15,6690,0317,3143,1,04
...,...,...,...,...,...,...,...,...
52930,N43,TP-DWC,18,7689,0118,5028,2,03
52931,N43,TP-DWC,31,14448,0101,7002,3,29
52932,N43,TP-DWC,30,13578,0101,4052,2,01
52933,N43,TZ-CHO3,1,0,0304,6048,3,05


In [113]:
conn = connect()
check_for_table(conn, 'Routes', routes.columns)
cursor = conn.cursor()
# Insert Dataframe into SQL Server:
for index, row in routes.iterrows():
    cursor.execute("INSERT INTO Routes (linia, kod, enroute_nr, odleglosc, ulica_id, nr_zespolu, typ, nr_przystanku) values(?,?,?,?,?,?,?,?)",
        (row['linia'], row['kod'], row['enroute_nr'], row['odleglosc'], row['ulica_id'], row.nr_zespolu, row.typ, row.nr_przystanku))
conn.commit()
cursor.close()
conn.close()

CREATE TABLE Routes (linia varchar(255),
kod varchar(255),
enroute_nr varchar(255),
odleglosc varchar(255),
ulica_id varchar(255),
nr_zespolu varchar(255),
typ varchar(255),
nr_przystanku varchar(255))
Table Routes created successfully.


In [114]:
# pozycje przystanków autobusowych
url = f'https://api.um.warszawa.pl/api/action/dbstore_get?id=ab75c33d-3a26-4342-b36a-6e5fef0a3ac3&apikey={apikey}'

response = requests.get(url)
if response.status_code == 200:
    print('ok')
    # Odpowiedź jest udana, można przetwarzać dane
    data = response.json()  # Jeśli odpowiedź jest w formacie JSON

dictionaries = []
for i in range(len(data['result'])):
    dictionary = {}
    for j in range(len(data['result'][i]['values'])):
        # print(data['result'][i]['values'][j]['key'], data['result'][i]['values'][j]['value'])
        c = data['result'][i]['values'][j]['key']
        v = data['result'][i]['values'][j]['value']
        dictionary[c] = v
    dictionaries.append(dictionary)
stops = pd.DataFrame(dictionaries)
stops

ok


,zespol,slupek,nazwa_zespolu,id_ulicy,szer_geo,dlug_geo,kierunek,obowiazuje_od
0,1001,01,Kijowska,2201,52.248455,21.044827,al.Zieleniecka,2023-01-02 00:00:00.0
1,1001,02,Kijowska,2201,52.249078,21.044443,Ząbkowska,2023-01-02 00:00:00.0
2,1001,03,Kijowska,2201,52.248928,21.044169,al.Zieleniecka,2023-01-02 00:00:00.0
3,1001,04,Kijowska,2201,52.249969,21.041588,Ząbkowska,2023-01-02 00:00:00.0
4,1001,05,Kijowska,1203,52.250319,21.043861,al.Zieleniecka,2023-01-02 00:00:00.0
...,...,...,...,...,...,...,...,...
7992,R-11,99,ZEA KLESZCZOWA,1221,52.196605,20.922746,ZEA KLESZCZOWA,2023-01-02 00:00:00.0
7993,R-13,00,ZEA STALOWA,2002,52.263590,21.047868,Stalowa,2023-01-02 00:00:00.0
7994,R-13,99,ZEA STALOWA,2002,52.263631,21.047922,ZEA STALOWA,2023-01-02 00:00:00.0
7995,R-19,00,WYDZIAŁ WŁOŚCIAŃSKA,2575,52.271236,20.968586,Włościańska,2023-01-02 00:00:00.0


In [117]:
conn = connect()
check_for_table(conn, 'Stops', stops.columns)
cursor = conn.cursor()
# Insert Dataframe into SQL Server:
for index, row in stops.iterrows():
    cursor.execute("INSERT INTO Stops (zespol, slupek,	nazwa_zespolu,	id_ulicy,	szer_geo,	dlug_geo,	kierunek,	obowiazuje_od) values(?,?,?,?,?,?,?,?)",
        (row['zespol'],	row['slupek'],	row['nazwa_zespolu'],	row['id_ulicy'],	row['szer_geo'],	row['dlug_geo'],	row['kierunek'],	row['obowiazuje_od']))
conn.commit()
cursor.close()
conn.close()

In [118]:
# słownik
url = f"https://api.um.warszawa.pl/api/action/public_transport_dictionary/?apikey={apikey}"
dfs = []
response = requests.get(url)
if response.status_code == 200:
    # Odpowiedź jest udana, można przetwarzać dane
    data = response.json()  # Jeśli odpowiedź jest w formacie JSON
    dictionaries = []
    for key in data['result'].keys():
        # print(data['result'][i]['values'][j]['key'], data['result'][i]['values'][j]['value'])
        for c, v in data['result'][key].items():
            dictionary = {}
            dictionary['type'] = key
            dictionary['value'] = v
            dictionary['id'] = c
            dictionaries.append(dictionary)
    
dictionary = pd.DataFrame(dictionaries)
dictionary

,type,value,id
0,ulice,Gerbera,0835
1,ulice,Gąsiorowskiego,0834
2,ulice,Gromali,0837
3,ulice,Głowackiego,0836
4,ulice,Galla Anonima,0831
...,...,...,...
1821,miejsca,NOWY DZIEKANÓW,NÓ
1822,miejsca,ALEKSANDRÓW,AL
1823,miejsca,ANTONINÓW,AN
1824,miejsca,AUGUSTÓWEK,AU


In [119]:
conn = connect()
check_for_table(conn, 'Dict', dictionary.columns)
cursor = conn.cursor()
# Insert Dataframe into SQL Server:
for index, row in dictionary.iterrows():
    cursor.execute("INSERT INTO Dict (type, value, id) values(?,?,?)",
        (row.type, row.value, row.id))
conn.commit()
cursor.close()
conn.close()

CREATE TABLE Dict (type varchar(255),
value varchar(255),
id varchar(255))
Table Dict created successfully.


 currentposition  
 dictionary  
 routes  
 schedule
 stopcode  
 stopinfo  
 stops

# zabawa danymi

In [121]:
schedule.merge(routes, left_on = 'trasa', right_on = 'kod', how = 'inner')

,zespol,slupek,linia_x,symbol_2,symbol_1,brygada,kierunek,trasa,czas,linia_y,kod,enroute_nr,odleglosc,ulica_id,nr_zespolu,typ,nr_przystanku
0,4009,01,154,null,null,1,rondo Zesłańców Syberyjskich,TP-RZS,04:27:00,154,TP-RZS,20,9046,2028,4043,1,05
1,4009,01,154,null,null,1,rondo Zesłańców Syberyjskich,TP-RZS,04:27:00,154,TP-RZS,21,9714,0101,4044,4,02
2,4009,01,154,null,null,1,rondo Zesłańców Syberyjskich,TP-RZS,04:27:00,154,TP-RZS,22,10198,0101,4045,7,71
3,4009,01,154,null,null,1,rondo Zesłańców Syberyjskich,TP-RZS,04:27:00,154,TP-RZS,1,0,0112,4015,3,09
4,4009,01,154,null,null,1,rondo Zesłańców Syberyjskich,TP-RZS,04:27:00,154,TP-RZS,3,1434,2013,4012,2,05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96301469,6061,05,17,null,t,22,Tarchomin Kościelny,TD-4TAK,06:54:00,2,TD-4TAK,4,2740,1992,1122,1,04
96301470,6061,05,17,null,t,22,Tarchomin Kościelny,TD-4TAK,06:54:00,2,TD-4TAK,7,4225,2108,1277,1,06
96301471,6061,05,17,null,t,22,Tarchomin Kościelny,TD-4TAK,06:54:00,2,TD-4TAK,6,3569,2108,1256,1,06
96301472,6061,05,17,null,t,22,Tarchomin Kościelny,TD-4TAK,06:54:00,2,TD-4TAK,9,5135,2108,1507,4,02


In [ ]:
tmp.loc[ ( tmp['linia'] == '1')  & (tmp['kod'] == 'TD-4AN03')]

,linia,kod,nr przystanku na trasie,odleglosc,ulica_id,nr_zespolu,typ,nr_przystanku
42810,1,TD-4AN03,24,10877,1701,1077,1,04
42811,1,TD-4AN03,25,11478,1701,1078,1,04
42812,1,TD-4AN03,26,12009,1701,1079,1,04
42813,1,TD-4AN03,27,12417,1567,1080,1,04
42814,1,TD-4AN03,20,8963,1532,1164,1,04
42815,1,TD-4AN03,21,9442,2001,1006,1,04
42816,1,TD-4AN03,22,9660,2001,1006,1,08
42817,1,TD-4AN03,23,10310,2001,1075,1,04
42818,1,TD-4AN03,28,13194,1901,1082,1,02
42819,1,TD-4AN03,29,13537,1901,1083,1,04


# google API

In [29]:
destination = "52.1968,20.9209"
origin = "52.248455,21.044827"
url = "https://maps.googleapis.com/maps/api/distancematrix/json" + f"?destinations={destination}" +  f"&origins={origin}" + f"&key={GOOGLE_KEY}"
response = requests.get(url)
if response.status_code == 200:
    print('ok')
    # Odpowiedź jest udana, można przetwarzać dane
    data = response.json()  # Jeśli odpowiedź jest w formacie JSON


ok


In [30]:
for key, val in data.items():
    print(key, val)

destination_addresses ['Przerwana 2, 02-484 Warszawa, Poland']
origin_addresses ['Targowa 7, 03-727 Warszawa, Poland']
rows [{'elements': [{'distance': {'text': '11.7 km', 'value': 11740}, 'duration': {'text': '22 mins', 'value': 1342}, 'status': 'OK'}]}]
status OK
